In [ ]:
%load_ext lab_black

In [ ]:
import pandas as pd

url = "https://www.data.gouv.fr/fr/datasets/r/bff6e5d3-c8eb-4f85-bf17-fba84b88fa71"
df = pd.read_csv(url, sep=";")

In [ ]:
import geopandas as gpd

github_url = "https://raw.githubusercontent.com/{user}/{repo}/master/{path}"

communes_fr = gpd.GeoDataFrame.from_file(
    github_url.format(
        user="gregoiredavid",
        repo="france-geojson",
        path="communes-version-simplifiee.geojson",
    )
)

In [ ]:
complete = (
    df.groupby("libelle_commune")
    .agg(dict(annee="max"))
    .reset_index()
    .merge(df, on=["libelle_commune", "annee"])
    .merge(
        communes_fr.query('code.str.startswith("2A") or code.str.startswith("2B")'),
        left_on="libelle_commune",
        right_on="nom",
    )
)

In [ ]:
complete.orientation_technico_economique_de_la_commune.unique()

In [ ]:
complete.loc[
    complete.orientation_technico_economique_de_la_commune
    == "Viticulture (appellation et autre)",
    "orientation_technico_economique_de_la_commune",
] = "wine"
complete.loc[
    complete.orientation_technico_economique_de_la_commune
    == "Fruits et autres cultures permanentes",
    "orientation_technico_economique_de_la_commune",
] = "crops"
complete.loc[
    complete.orientation_technico_economique_de_la_commune
    == "Cultures générales  (autres grandes cultures)",
    "orientation_technico_economique_de_la_commune",
] = "crops"
complete.loc[
    complete.orientation_technico_economique_de_la_commune
    == "Fleurs et horticulture diverse",
    "orientation_technico_economique_de_la_commune",
] = "crops"
complete.loc[
    complete.orientation_technico_economique_de_la_commune == "Ovins et caprins",
    "orientation_technico_economique_de_la_commune",
] = "cattle"
complete.loc[
    complete.orientation_technico_economique_de_la_commune == "Bovins viande",
    "orientation_technico_economique_de_la_commune",
] = "cattle"
complete.loc[
    complete.orientation_technico_economique_de_la_commune == "Porcins",
    "orientation_technico_economique_de_la_commune",
] = "cattle"
complete.loc[
    complete.orientation_technico_economique_de_la_commune == "Autres herbivores",
    "orientation_technico_economique_de_la_commune",
] = "cattle"
complete.loc[
    complete.orientation_technico_economique_de_la_commune == "Granivores mixtes",
    "orientation_technico_economique_de_la_commune",
] = "cattle"
complete.loc[
    complete.orientation_technico_economique_de_la_commune
    == "Polyculture et polyélevage",
    "orientation_technico_economique_de_la_commune",
] = "mixed"
complete.loc[
    complete.orientation_technico_economique_de_la_commune.isnull(),
    "orientation_technico_economique_de_la_commune",
] = "no data"

In [ ]:
def random_points_in_polygon(poly, num_points):
    min_x, min_y, max_x, max_y = poly.bounds

    points = []

    while len(points) < num_points:
        random_point = Point(
            [random.uniform(min_x, max_x), random.uniform(min_y, max_y)]
        )
        if random_point.within(poly):
            points.append(random_point)

    return MultiPoint(
        sum(
            (list(p.coords) for p in points),
            [],
        )
    )


geo = list(
    random_points_in_polygon(elt.geometry, elt.exploitations_agricoles)
    for _, elt in complete.iterrows()
)

In [ ]:
c

In [ ]:
## Generate the elevation map below

In [ ]:
import rasterio
from rasterio.plot import show
from rasterio.merge import merge

directory = Path(
    "/home/xo/Documents/data/SIG/BDALTIV2_2-0_75M_ASC_LAMB93-IGN69_FRANCE_2018-01-15/BDALTIV2/1_DONNEES_LIVRAISON_2018-01-00245/BDALTIV2_MNT_75M_ASC_LAMB93_IGN69_FRANCE/"
)

list_ = [
    "BDALTIV2_75M_FXX_1125_6075_MNT_LAMB93_IGN69.asc",
    "BDALTIV2_75M_FXX_1125_6150_MNT_LAMB93_IGN69.asc",
    "BDALTIV2_75M_FXX_1125_6225_MNT_LAMB93_IGN69.asc",
    "BDALTIV2_75M_FXX_1200_6075_MNT_LAMB93_IGN69.asc",
    "BDALTIV2_75M_FXX_1200_6150_MNT_LAMB93_IGN69.asc",
    "BDALTIV2_75M_FXX_1200_6225_MNT_LAMB93_IGN69.asc",
    "BDALTIV2_75M_FXX_1200_6300_MNT_LAMB93_IGN69.asc",
]


sources = list(rasterio.open(directory / filename) for filename in list_)
dest_array, out_transform = merge(sources)
# dest_array[dest_array == -99999.0] = 0

In [ ]:
xmin, ymax = out_transform.xoff, out_transform.yoff
resolution = 75
xmax = xmin + dest_array.shape[2] * resolution
ymin = ymax - dest_array.shape[1] * resolution

x = np.linspace(xmin, xmax, dest_array.shape[2])
y = np.linspace(ymin, ymax, dest_array.shape[1])

In [ ]:
from cartes.crs import Lambert93, PlateCarree

fig, ax = plt.subplots(figsize=(5, 10), subplot_kw=dict(projection=Lambert93()))
ret = ax.contour(
    x,
    y,
    dest_array[0][::-1, :],
    levels=(0, 500, 1000, 1500, 2500, 3000),
    cmap="Greys",
)

In [ ]:
from shapely.geometry import LineString, MultiLineString

contours = {
    "geometry": list(
        MultiLineString(list(s for s in segs if s.shape[0] > 2)).simplify(1e3)
        for segs in ret.allsegs
    ),
    "level": list(ret.levels),
}
c = GeoDataFrame.from_dict(contours)

In [ ]:
chart = (
    alt.hconcat(
        alt.Chart(c.set_crs(2154).to_crs(4326))
        .mark_geoshape(opacity=0.3)
        .encode(alt.Color("level:Q", legend=None, scale=alt.Scale(scheme="browns")))
        .properties(
            width=200,
            height=500,
            title="Elevation from IGN BD ALTI®",
        )
        .project(
            "conicConformal", rotate=[-3, -46.5], parallels=[49, 44], pointRadius=2
        ),
        alt.Chart(GeoDataFrame(complete.assign(geometry=geo)))
        .mark_geoshape(opacity=0.8)
        .encode(
            alt.Color(
                "orientation_technico_economique_de_la_commune",
                legend=alt.Legend(title="", orient="bottom"),
                # title="",
                scale=alt.Scale(
                    domain=["cattle", "crops", "mixed", "wine"],
                    range=["#c79353", "#767528", "#95815c", "#522f60"],
                ),
            ),
            alt.Tooltip("libelle_commune"),
        )
        .project(
            "conicConformal", rotate=[-3, -46.5], parallels=[49, 44], pointRadius=2
        )
        .properties(
            width=200,
            height=500,
            title="and agriculture in Corsica",
        ),
    )
    .configure_title(font="Fira Sans", fontSize=18, anchor="start")
    .configure_legend(labelFont="Fira Sans", labelFontSize=13)
    .configure_view(stroke=None)
)
chart

In [ ]:
chart.save("../contributions/challenge_day16.svg")